# Capstone Project: Create a Customer Segmentation Report for Arvato Financial Services

In this project, you will analyze demographics data for customers of a mail-order sales company in Germany, comparing it against demographics information for the general population. You'll use unsupervised learning techniques to perform customer segmentation, identifying the parts of the population that best describe the core customer base of the company. Then, you'll apply what you've learned on a third dataset with demographics information for targets of a marketing campaign for the company, and use a model to predict which individuals are most likely to convert into becoming customers for the company. The data that you will use has been provided by our partners at Bertelsmann Arvato Analytics, and represents a real-life data science task.

If you completed the first term of this program, you will be familiar with the first part of this project, from the unsupervised learning project. The versions of those two datasets used in this project will include many more features and has not been pre-cleaned. You are also free to choose whatever approach you'd like to analyzing the data rather than follow pre-determined steps. In your work on this project, make sure that you carefully document your steps and decisions, since your main deliverable for this project will be a blog post reporting your findings.

In [1]:
# !pip install modin[dask]

In [13]:
# import libraries here; add more as necessary
import numpy as np
import pandas as pd
# import modin.pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB


# from imblearn.over_sampling import SMOTENC
# from imblearn.over_sampling import SMOTE

sns.set(style="ticks", color_codes=True)


# magic word for producing visualizations in notebook
%matplotlib inline

AttributeError: module 'numpy' has no attribute '__version__'

## Part 0: Get to Know the Data

There are four data files associated with this project:

- `Udacity_AZDIAS_052018.csv`: Demographics data for the general population of Germany; 891 211 persons (rows) x 366 features (columns).
- `Udacity_CUSTOMERS_052018.csv`: Demographics data for customers of a mail-order company; 191 652 persons (rows) x 369 features (columns).
- `Udacity_MAILOUT_052018_TRAIN.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 982 persons (rows) x 367 (columns).
- `Udacity_MAILOUT_052018_TEST.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 833 persons (rows) x 366 (columns).

Each row of the demographics files represents a single person, but also includes information outside of individuals, including information about their household, building, and neighborhood. Use the information from the first two files to figure out how customers ("CUSTOMERS") are similar to or differ from the general population at large ("AZDIAS"), then use your analysis to make predictions on the other two files ("MAILOUT"), predicting which recipients are most likely to become a customer for the mail-order company.

The "CUSTOMERS" file contains three extra columns ('CUSTOMER_GROUP', 'ONLINE_PURCHASE', and 'PRODUCT_GROUP'), which provide broad information about the customers depicted in the file. The original "MAILOUT" file included one additional column, "RESPONSE", which indicated whether or not each recipient became a customer of the company. For the "TRAIN" subset, this column has been retained, but in the "TEST" subset it has been removed; it is against that withheld column that your final predictions will be assessed in the Kaggle competition.

Otherwise, all of the remaining columns are the same between the three data files. For more information about the columns depicted in the files, you can refer to two Excel spreadsheets provided in the workspace. [One of them](./DIAS Information Levels - Attributes 2017.xlsx) is a top-level list of attributes and descriptions, organized by informational category. [The other](./DIAS Attributes - Values 2017.xlsx) is a detailed mapping of data values for each feature in alphabetical order.

In the below cell, we've provided some initial code to load in the first two datasets. Note for all of the `.csv` data files in this project that they're semicolon (`;`) delimited, so an additional argument in the [`read_csv()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) call has been included to read in the data properly. Also, considering the size of the datasets, it may take some time for them to load completely.

You'll notice when the data is loaded in that a warning message will immediately pop up. Before you really start digging into the modeling and analysis, you're going to need to perform some cleaning. Take some time to browse the structure of the data and look over the informational spreadsheets to understand the data values. Make some decisions on which features to keep, which features to drop, and if any revisions need to be made on data formats. It'll be a good idea to create a function with pre-processing steps, since you'll need to clean all of the datasets before you work with them.

In [ ]:
# load in the data
# azdias = pd.read_csv('Udacity_AZDIAS_052018.csv', sep=';')
# customers = pd.read_csv('Udacity_CUSTOMERS_052018.csv', sep=';')

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

The following meanings are equivalent of NaN (by looking at DIAS Attributes - Values 2017.xlsx)- 
1. The value of -1, represents an unknown value
2. For a few other attrbutes a value of 0 or 9 or 10 is used to represent an unknow value... we'll takle this later


## Part 2: Supervised Learning Model

Now that you've found which parts of the population are more likely to be customers of the mail-order company, it's time to build a prediction model. Each of the rows in the "MAILOUT" data files represents an individual that was targeted for a mailout campaign. Ideally, we should be able to use the demographic information from each individual to decide whether or not it will be worth it to include that person in the campaign.

The "MAILOUT" data has been split into two approximately equal parts, each with almost 43 000 data rows. In this part, you can verify your model with the "TRAIN" partition, which includes a column, "RESPONSE", that states whether or not a person became a customer of the company following the campaign. In the next part, you'll need to create predictions on the "TEST" partition, where the "RESPONSE" column has been withheld.

In [ ]:
mailout_train = pd.read_csv('Udacity_MAILOUT_052018_TRAIN.csv', sep=';')

In [ ]:
print(mailout_train.shape)
print((mailout_train.RESPONSE.sum())*100/len(mailout_train))

# Observations on data

1. The percentage of folks who actually become a customer is very less... hence we need a way to score with a higher recall

In [ ]:
df_attrs = pd.read_excel('DIAS Attributes - Values 2017.xlsx', skiprows=1)
df_values = df_attrs.copy()
df_attrs.drop(columns=df_attrs.columns[0], inplace=True)
df_attrs.dropna(inplace=True)
df_attrs = df_attrs.reset_index()
print(df_attrs.shape)
df_attrs.head()

In [ ]:
df_values.drop(columns=df_values.columns[0], inplace=True)
df_values.Attribute = df_values.Attribute.fillna(method='ffill')
print(df_values.shape)
df_values.head()

In [ ]:
common_columns = (set(list(df_attrs.Attribute)) & set(list(mailout_train.columns)))
common_columns

In [ ]:
# a perusal of the data dictionary gives us the following info:
1. Most of the features that do not have "..." in the Value column are categorical in nature, either because they are binned, ordinal or nominal in nature
2. The exception to the above rule are the columns: ALTER_HH.. which is the age

In [ ]:
def mark_invalid_values_as_null(df):
    categorical_dict = {}
    df_categorical = pd.DataFrame()
    columns_with_unknown_values = ''
    for colname in common_columns:
#         print('{} - {}'.format(colname, df_attrs[df_attrs.Attribute==colname].Type.values[0]))
        valid_values = df_values[(df_values.Attribute==colname) & \
                                 ((df_values.Meaning.str.contains('unknown')==False) \
                                 | df_values.Meaning.isnull())].Value.values
        if ('X' in df[colname].unique()) or ('XX' in df[colname].unique()):
            print('unique values for column:{}, are {}'.format(colname, df[colname].unique()))
        if (df_attrs[df_attrs.Attribute==colname].Value.values[0] == '…'): # continuous variable
            print('{} is a continuous feature'.format(colname))
        else:
            if df_attrs[df_attrs.Attribute==colname].Meaning.values[0].find('unknown') > -1:
                unknown_values = str(df_attrs[df_attrs.Attribute==colname].Value.values[0])
                print('unknown values for {} are {}'.format(colname, unknown_values))
            m = df[colname].isin(valid_values)
            df[colname] = df[colname].where(m, other=np.nan)
            if (len(m) -sum(m)) > 0:
                columns_with_unknown_values += "'" + colname + "'" + ','
            print('nulls + unknowns = {}'.format(len(m) -sum(m)))
        print('-----------------------------------')
    return df, categorical_dict, columns_with_unknown_values

In [ ]:
mailout_train1, _, columns_with_unknowns = mark_invalid_values_as_null(mailout_train[common_columns])
# column:CAMEO_DEUG_2015

In [ ]:
# Let's replace all X and XX with NaN
mailout_train1 = mailout_train1.replace('X', np.nan)
mailout_train1 = mailout_train1.replace('XX', np.nan)

In [5]:
def plot_hist_missing_col(df):
    '''
    INPUT:
        df - (dataframe), dataframe to check for missing data and plot histogram of the missing data in columns
    OUTPUT:
        visualization: (histogram), of the missing data
    '''
    plt.hist(df['pct'], bins=20, facecolor='g', alpha=0.75)

    plt.xlabel('Pe\rcentage of missing value (%)', fontsize=12)
    plt.ylabel('Counts', fontsize=12)
    plt.title('Histogram of missing value counts in columns', fontsize=12)
    plt.grid(True)
    plt.show()


def get_null_counts(df):
    null_data = df.isnull().sum()[df.isnull().sum() > 0]
    dict = {'nulls': null_data.values, 'pct': np.round(null_data.values*100/df.shape[0], 2)}
    df_nulls = pd.DataFrame(data=dict, index=null_data.index)
    return df_nulls

In [6]:
mailout_nulls = get_null_counts(mailout_train1)
plot_hist_missing_col(mailout_nulls)
# mailout_nulls.sort_values(by=['pct'], axis=0, kind='quicksort', ascending=False)
# mailout_nulls
print(mailout_nulls[mailout_nulls.pct > 24])

NameError: name 'mailout_train1' is not defined

In [7]:
# Observations
# The columns that have a large number of nulls pretty much match the columns of general population, having large number of nulls
# Also all the columns that were used for clustering are available in the mailout dataset... so we should be able to apply the same cleanup process to this dataset
mailout_train1 = mailout_train1.drop(columns=mailout_nulls[mailout_nulls.pct>24].index)
final_column_list = mailout_train1.columns


NameError: name 'mailout_train1' is not defined

In [8]:
# The following columns needs to be made category types
# so that they can easily be consumed by SimpleImputer and later by getdummies appropriately
# 'CAMEO_DEU_2015', 'west'
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
# fitted = le.fit_transform(mailout_train1[['OST_WEST_KZ']])

fit_by = pd.Series([i for i in mailout_train1['OST_WEST_KZ'].unique() if type(i) == str])
le.fit(fit_by)
mailout_train1["OST_WEST_KZ"] = fit_by.apply(lambda x: le.transform([x])[0] if type(x) == str else x)

fit_by = pd.Series([i for i in mailout_train1['CAMEO_DEU_2015'].unique() if type(i) == str])
le.fit(fit_by)
mailout_train1["CAMEO_DEU_2015"] = fit_by.apply(lambda x: le.transform([x])[0] if type(x) == str else x)

AttributeError: module 'numpy' has no attribute '__config__'

In [ ]:
mailout_train1["OST_WEST_KZ"].unique()

In [9]:
# get the list of continuous variables
# df_attrs[df_attrs.Attribute==colname].Value.values[0] == '…'
continuous_variables = df_attrs[df_attrs.Value == '…'].Attribute
mailout_train1[continuous_variables].hist(figsize=(10,10))

NameError: name 'df_attrs' is not defined

In [10]:
# its clear from the above that there is usually one value that is most commom by a big margin
mailout_train1.GEBURTSJAHR.value_counts()
# let's convert the GEBURTSJAHR to age
mailout_train1.GEBURTSJAHR = 2020 - mailout_train1.GEBURTSJAHR
mailout_train1.GEBURTSJAHR.value_counts()
# for this column it miht make sense to take the mean age to replace the .. but since it has no nulls... let's skiip this part for now

NameError: name 'mailout_train1' is not defined

In [ ]:
null_row_counts = mailout_train.isnull().sum(axis=1)
null_row_counts.values

plt.hist(null_row_counts.values, bins=40, facecolor='g', alpha=0.75)

plt.xlabel('Number of missing columns', fontsize=12)
plt.ylabel('Counts', fontsize=12)
plt.title('Histogram of missing column value counts per row', fontsize=12)
plt.grid(True)
plt.show()

In [ ]:
null_row_counts

In [ ]:
# there are rows that have more than 50-100-200 columns with null values... fillng up nan's for these rows is unlikely to be meaningful
mailout_train1['RESPONSE'] = mailout_train.RESPONSE
print(mailout_train1.RESPONSE.sum())
print(mailout_train1.dropna(thresh=len(mailout_train1.columns)-40).shape)
print(mailout_train1.dropna(thresh=len(mailout_train1.columns)-40).RESPONSE.sum())
mailout_train2 = mailout_train1.dropna(thresh=len(mailout_train1.columns)-40)

In [ ]:
mailout_train2 = mailout_train2.reset_index(drop=True)

In [ ]:
df_response = mailout_train2.RESPONSE
mailout_train2 = mailout_train2.drop(columns='RESPONSE')

In [ ]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values= np.nan, strategy='most_frequent')

In [ ]:
imputed_data = imputer.fit_transform(mailout_train2)
mailout_train2 = pd.DataFrame(imputed_data, columns = mailout_train2.columns)

In [ ]:
# get categorical columns
category_columns = df_attrs[df_attrs.Value != '…'].Attribute
category_columns = set(category_columns).intersection(mailout_train2.columns)
category_columns

In [ ]:
mailout_train_encoded = pd.get_dummies(mailout_train2, columns=category_columns, prefix=category_columns)

In [ ]:
mailout_train_encoded.shape

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
mailout_scaled = scaler.fit_transform(mailout_train_encoded)
mailout_scaled = pd.DataFrame(mailout_scaled, columns= list(mailout_train_encoded.columns.values))

In [ ]:
mailout_scaled.shape

In [ ]:
import time
def show_confusion_matrix(classifier, X_test, y_test):
    class_names=['NO', 'YES']
    # Plot non-normalized confusion matrix
    titles_options = [("Confusion matrix, without normalization", None),
                      ("Normalized confusion matrix", 'true')]
    for title, normalize in titles_options:
        disp = plot_confusion_matrix(classifier, X_test, y_test,
                                     display_labels=class_names,
                                     cmap=plt.cm.Blues,
                                     normalize=normalize)
        disp.ax_.set_title(title)

        print(title)
        print(disp.confusion_matrix)

    plt.show()

def fit_classifier(clf, param_grid, X_train, Y_train, X_test, Y_test):
    """
    Fits a classifier to its training data using GridSearchCV and calculates ROC AUC score
    
    INPUT:
    - clf (classifier): classifier to fit
    - param_grid (dict): classifier parameters used with GridSearchCV
    - X_train (DataFrame): training input
    - y_train (DataFrame): training output
            
    OUTPUT:
    - classifier: input classifier fitted to the training data
    """
    
    # cv uses StratifiedKFold
    # scoring roc_auc available as parameter
    start = time.time()
    grid = GridSearchCV(estimator=clf, param_grid=param_grid, scoring='roc_auc', cv=5, verbose=2, n_jobs=4)
    print("Training {} :".format(clf.__class__.__name__))
    grid.fit(X_train, Y_train)
    end = time.time()
    time_taken = round(end-start,2)

    print(clf.__class__.__name__)
    print("Time taken : {} secs".format(time_taken))
    print("Best score : {}".format(round(grid.best_score_,4)))
    print("*"*40)
    
    show_confusion_matrix(grid.best_estimator_, X_test, Y_test)
    
    print('--------------------------------')
    
    return grid.best_score_, grid.best_estimator_, time_taken

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(mailout_scaled.astype(float), \
                                                    df_response, \
                                                    test_size=0.33, random_state=40)

In [ ]:
y_train.sum()

In [ ]:
# # let's get a quick feel of predictability
# grid_params = {
#     'class_weight': ['balanced'],
#     'n_estimators': [100,300,600,1000]
# }

# fit_classifier(RandomForestClassifier(), grid_params, X_train, y_train, X_test, y_test)

In [ ]:
# time to try NB.. but we'll have to do PCA so that we get relatively linear independent features
mailout_scaled.shape

In [ ]:
from sklearn.decomposition import PCA

# let's create a reusable function to execute PCA on data to a stated n number of components
# because we'll need to experiment with different number of PCA components
def do_pca(data, n):
    pca = PCA(n_components=n, random_state=1)
    X_pca = pca.fit_transform(data)
    return pca, X_pca

def plot_variance(pca):
    num_components=len(pca.explained_variance_ratio_)
    num_components = np.arange(num_components)
    ratios = pca.explained_variance_ratio_
 
    plt.figure(figsize=(10, 10))
    ax = plt.subplot(111)
    cumvals = np.cumsum(ratios)
    ax.bar(num_components, ratios)
    ax.plot(num_components, cumvals)
    
    ax.grid()
    ax.set_axisbelow(True)
    ax.grid(linewidth='0.5', alpha=0.5)
   
 
    ax.xaxis.set_tick_params(width=0)
    ax.yaxis.set_tick_params(width=2, length=12)
 
    ax.set_xlabel("Principal Component")
    ax.set_ylabel("Variance Explained (%)")
    plt.title('Explained Variance Per Principal Component')

In [ ]:
pca, X_pca = do_pca(mailout_scaled, 700)
plot_variance(pca)

In [ ]:
# the rate of increase in variance covered decreases after around 200 components.
# Let's derive 200 components from this
pca, X_pca = do_pca(mailout_scaled, 200)
plot_variance(pca)

In [ ]:
X_train3, X_test3, y_train3, y_test3 = train_test_split(X_pca, \
                                                    df_response, \
                                                    test_size=0.33, random_state=40)
fit_classifier(GaussianNB(), grid_params, X_train3, y_train3, X_test3, y_test3)

## Part 3: Kaggle Competition

Now that you've created a model to predict which individuals are most likely to respond to a mailout campaign, it's time to test that model in competition through Kaggle. If you click on the link [here](http://www.kaggle.com/t/21e6d45d4c574c7fa2d868f0e8c83140), you'll be taken to the competition page where, if you have a Kaggle account, you can enter. If you're one of the top performers, you may have the chance to be contacted by a hiring manager from Arvato or Bertelsmann for an interview!

Your entry to the competition should be a CSV file with two columns. The first column should be a copy of "LNR", which acts as an ID number for each individual in the "TEST" partition. The second column, "RESPONSE", should be some measure of how likely each individual became a customer – this might not be a straightforward probability. As you should have found in Part 2, there is a large output class imbalance, where most individuals did not respond to the mailout. Thus, predicting individual classes and using accuracy does not seem to be an appropriate performance evaluation method. Instead, the competition will be using AUC to evaluate performance. The exact values of the "RESPONSE" column do not matter as much: only that the higher values try to capture as many of the actual customers as possible, early in the ROC curve sweep.

In [ ]:
# mailout_test = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TEST.csv', sep=';')